Rete per la gestione di dei dati con shape [32,768] tramite convoluzione 1D

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

/home/antoniodeblasi/anaconda3/envs/bioInfo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

In [3]:
import torch

class MyDataSet(Dataset):
    def __init__(self, path):
        self.path = path
        self.df_sequences = os.listdir(path+'/embeddings')
        self.df_labels = os.listdir(path+'/labels')

    def __len__(self):
        return len(self.df_sequences)

    def __getitem__(self, index):
        sequence = torch.load(self.path+'/embeddings/'+self.df_sequences[index])#.swapaxes(1,2)
  
        sequence = torch.mean(sequence, dim=1) #[32, 138,768] --> [32, 768]
        sequence = sequence.unsqueeze(dim=1)   #[32,768] --> [32,1,768] for the convolution
        label = torch.load(self.path+'/labels/'+self.df_labels[index]).float()
        return sequence, label

In [4]:
DataTrainpath="/home/antoniodeblasi/Scaricati/Dataset_V1"
DataTestpath="/home/antoniodeblasi/Scaricati/Dataset_V1_testing"
DataValpath="/home/antoniodeblasi/Scaricati/Dataset_V1_validation"

#/admin:///ai4bioinfo2023/project_15/Training_N/DatasetN

# Create custom dataset object
train_data_object = MyDataSet(DataTrainpath)
test_data_object = MyDataSet(DataTestpath)
val_data_object = MyDataSet(DataValpath)

def collate(batch):
  return batch[0]



In [5]:
for seq, _ in torch.utils.data.DataLoader(train_data_object,
        batch_size=1, shuffle = False, collate_fn=collate):
    print(seq.shape)
    break

torch.Size([32, 1, 768])


In [6]:
import pytorch_lightning as pl

In [7]:
class MyDataModule(pl.LightningDataModule):

  def setup(self, stage):
    self.dataset = ""#MyDataSet("")


  def train_dataloader(self):
    return torch.utils.data.DataLoader(train_data_object,
        batch_size=1, shuffle = False, collate_fn=collate)
  def val_dataloader(self):
    return torch.utils.data.DataLoader(val_data_object,
        batch_size=1, shuffle = False, collate_fn=collate)
  def test_dataloader(self):
    return torch.utils.data.DataLoader(test_data_object,
       batch_size=1, shuffle = False, collate_fn=collate)



In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics

class CNN(pl.LightningModule):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Pooling globale per ridurre le dimensioni

        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
        # for validation/testing
        self.accuracy = torchmetrics.Accuracy(task="binary")


    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.global_avg_pool(x)
        x = torch.relu(self.fc1(x.squeeze()))
        x = self.sigmoid(self.fc2(x))
        return x

    def cross_entropy_loss(self, logits, labels):
      return F.binary_cross_entropy(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x).squeeze()
        loss = self.cross_entropy_loss(logits, y)
        self.log('train_loss', loss)
        
        
        return loss


    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x).squeeze()
        loss = self.cross_entropy_loss(logits, y)
        acc = self.accuracy(logits,y)
        
        self.log('val_loss', loss)
        self.log('val_accuracy', acc)
        
        
        

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x).squeeze()
        loss = self.cross_entropy_loss(logits, y)
        accuracy = torchmetrics.Accuracy(task="binary").cuda()
        print(y.shape)
        print(logits.shape)
        train_accuracy = torch.sum(y == logits)/logits.shape[0]

        acc = accuracy(logits,y)
        
        self.log('test_loss', loss)
        self.log('test_accuracy', train_accuracy)

    def configure_optimizers(self):
      optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.8)
      return optimizer

# Creazione del modello
model = CNN()


In [72]:
x = torch.zeros(1,1,768)
y = mod(x)
print(y)

tensor([0.5346], grad_fn=<SigmoidBackward0>)


In [ ]:
for i, el in enumerate(test_data_object):

    x = el[0]
    print(mod(x))

In [33]:
mod = CNN.load_from_checkpoint("lightning_logs/version_45/checkpoints/epoch=39-step=30600.ckpt")


x = torch.randn(32,1,768)


trainer = pl.Trainer(max_epochs = 40)

data_module = MyDataModule()

p = trainer.test(mod, data_module)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/antoniodeblasi/anaconda3/envs/bioInfo/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0:   0%|          | 0/96 [00:00<?, ?it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   1%|          | 1/96 [00:00<00:00, 199.40it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   2%|▏         | 2/96 [00:00<00:00, 115.86it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   3%|▎         | 3/96 [00:00<00:00, 98.19it/s] torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   4%|▍         | 4/96 [00:00<00:01, 91.23it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   5%|▌         | 5/96 [00:00<00:01, 88.37it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   6%|▋         | 6/96 [00:00<00:01, 84.92it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   7%|▋         | 7/96 [00:00<00:01, 83.51it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   8%|▊         | 8/96 [00:00<00:01, 84.30it/s]torch.Size([32])
torch.Size([32])
Testing DataLoader 0:   9%|▉         | 9/96 [00:00<00:01, 82.17it/s]torch.Size([32])
to

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │            0.0            │
│         test_loss         │    0.6909841895103455     │
└───────────────────────────┴───────────────────────────┘

In [10]:


trainer = pl.Trainer(max_epochs = 40)

data_module = MyDataModule()

p = trainer.test(model, data_module)


trainer.fit(model, data_module)
# Valutazione del modello
p = trainer.test(model, data_module)
print("Loss sul set di validazione:", p)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 96/96 [00:01<00:00, 64.96it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │       0.5322265625        │
│         test_loss         │    0.6910873055458069     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params
------------------------------------------------------
0 | conv1           | Conv1d            | 128   
1 | conv2           | Conv1d            | 6.2 K 
2 | conv3           | Conv1d            | 24.7 K
3 | pool            | MaxPool1d         | 0     
4 | global_avg_pool | AdaptiveAvgPool1d | 0     
5 | fc1             | Linear            | 8.3 K 
6 | fc2             | Linear            | 65    
7 | sigmoid         | Sigmoid           | 0     
8 | accuracy        | BinaryAccuracy    | 0     
------------------------------------------------------
39.4 K    Trainable params
0         Non-trainable params
39.4 K    Total params
0.157     Total estimated model params size (MB)


Epoch 39: 100%|██████████| 765/765 [00:13<00:00, 57.47it/s, v_num=45]       

...

Epoch 39: 100%|██████████| 765/765 [00:13<00:00, 57.44it/s, v_num=45]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 96/96 [00:01<00:00, 66.96it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │       0.5322265625        │
│         test_loss         │    0.6909841895103455     │
└───────────────────────────┴───────────────────────────┘

Loss sul set di validazione: [{'test_loss': 0.6909841895103455, 'test_accuracy': 0.5322265625}]
